<a href="https://colab.research.google.com/github/yogeswarmalireddy/Bank-Analytics-MySQL/blob/main/Soccer_Player_Re_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install Ultralytics (YOLOv8/YOLOv11 library)
!pip install ultralytics
!pip install opencv-python-headless

# Clone your project repo (if it's on GitHub)
# Or manually upload files using the Colab file sidebar


In [8]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import OrderedDict
import math

# ---------------- Centroid Tracker ------------------
class CentroidTracker:
    def __init__(self, max_distance=50):
        self.nextObjectID = 1
        self.objects = OrderedDict()
        self.max_distance = max_distance

    def update(self, detections):
        new_objects = OrderedDict()

        for (x1, y1, x2, y2) in detections:
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            matched = False
            for obj_id, (ox, oy) in self.objects.items():
                distance = math.hypot(cx - ox, cy - oy)
                if distance < self.max_distance:
                    new_objects[obj_id] = (cx, cy)
                    matched = True
                    break

            if not matched:
                new_objects[self.nextObjectID] = (cx, cy)
                self.nextObjectID += 1

        self.objects = new_objects
        return self.objects

# ---------------- Main Detection + Tracking ------------------
model = YOLO("/content/best.pt")  # Make sure best.pt is in your folder

cap = cv2.VideoCapture("/content/15sec_input_720p.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

tracker = CentroidTracker()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, verbose=False)[0]
    detections = []

    for box in results.boxes.xyxy:
        x1, y1, x2, y2 = box[:4].cpu().numpy()
        detections.append((x1, y1, x2, y2))

    tracked_objects = tracker.update(detections)

    for (obj_id, (cx, cy)) in tracked_objects.items():
        cv2.putText(frame, f"Player {obj_id}", (cx - 10, cy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)

    out.write(frame)
    # Optional: cv2.imshow("Tracking", frame); if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
out.release()
cv2.destroyAllWindows()

print("✅ Output video saved as output.mp4 with Player IDs in order.")


✅ Output video saved as output.mp4 with Player IDs in order.
